In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
! pip install geopy

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [4]:
!pip install bs4
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=2168ac6e2b0ec60d7abb6fbfdf9723d955f2a701e9a4a1e764a039e1f9345e1d
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


In [5]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [6]:
data  = requests.get(url).text

In [7]:
soup = BeautifulSoup(data,"html5lib")

In [8]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [9]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [10]:
df.shape

(103, 3)

In [11]:
df["C"] = ""

In [12]:
df.head()

,PostalCode,Borough,Neighborhood,C
0,M3A,North York,Parkwoods,
1,M4A,North York,Victoria Village,
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",
3,M6A,North York,"Lawrence Manor, Lawrence Heights",
4,M7A,Queen's Park,Ontario Provincial Government,


In [13]:
df.shape

(103, 4)

In [14]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_63514e74a661494dbb2ff06589064bb7 = 'https://s3-api.us-geo.objectstorage.softlayer.net'
else:
    endpoint_63514e74a661494dbb2ff06589064bb7 = 'https://s3-api.us-geo.objectstorage.service.networklayer.com'

client_63514e74a661494dbb2ff06589064bb7 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='0oa9wH8UtO__AW95Ml0l-NP2JqG8hoSa3GSc1KG7NdIN',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_63514e74a661494dbb2ff06589064bb7)

body = client_63514e74a661494dbb2ff06589064bb7.get_object(Bucket='torontoneighborhoodanalysis-donotdelete-pr-lkgoh58sg3ubeh',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_X= pd.read_csv(body)
df_X.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Lets get the latitude and longitude of each borough and it's neighborhood based on the postal code

In [15]:
df_X.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)
Toronto_merged = df
Toronto_merged  = Toronto_merged.join(df_X.set_index('PostalCode'), on='PostalCode')
Toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,C,Latitude,Longitude
0,M3A,North York,Parkwoods,,43.753259,-79.329656
1,M4A,North York,Victoria Village,,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,,43.662301,-79.389494


#### Lets analyze and visualize the North York and it's neighborhood 

In [16]:
NorthYork_data=Toronto_merged[Toronto_merged['Borough']=='North York']
NorthYork_data.head()

,PostalCode,Borough,Neighborhood,C,Latitude,Longitude
0,M3A,North York,Parkwoods,,43.753259,-79.329656
1,M4A,North York,Victoria Village,,43.725882,-79.315572
3,M6A,North York,"Lawrence Manor, Lawrence Heights",,43.718518,-79.464763
7,M3B,North York,Don Mills North,,43.745906,-79.352188
10,M6B,North York,Glencairn,,43.709577,-79.445073


In [17]:
NorthYork_data.shape

(24, 6)

#### Calculating the latitude and longitude of the North york, Toronto using geopy library

In [57]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
address = 'North York, Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York, TN are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York, TN are 43.7543263, -79.44911696639593.


In [19]:
! pip install folium
import folium 

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 4.3 MB/s  eta 0:00:01


#### Visualizing the Norh york, Toronto using the folium library 

In [20]:
# create map of Norh york using latitude and longitude values
map_NorthYork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(NorthYork_data['Latitude'], NorthYork_data['Longitude'], NorthYork_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NorthYork)  
    
map_NorthYork

#### Define Foursquare Credentials and Version

In [21]:
CLIENT_ID = 'AIM4IHE2EKM1QH3VTP2E1BIA3CK1BB1M4OWC441WQR1XEKTQ' # your Foursquare ID
CLIENT_SECRET = '0ZRMVQMS5V1NBOPPTA1GYM3X54OGBAB1APU0LQJRKBUMPXNP' # your Foursquare Secret
ACCESS_TOKEN = 'TD2IJU0O2AZCK32F25CLW1OLPCVRYRJ5OUQKXTNCB4IN4IO3' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AIM4IHE2EKM1QH3VTP2E1BIA3CK1BB1M4OWC441WQR1XEKTQ
CLIENT_SECRET:0ZRMVQMS5V1NBOPPTA1GYM3X54OGBAB1APU0LQJRKBUMPXNP


#### Let's explore the first neighborhood in our dataframe.

In [22]:
NorthYork_data.loc[0, 'Neighborhood']

'Parkwoods'

In [23]:
neighborhood_latitude = NorthYork_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = NorthYork_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = NorthYork_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


#### Now, let's get the top 100 venues that are in Parkwoods within a radius of 500 meters.¶

In [24]:
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=AIM4IHE2EKM1QH3VTP2E1BIA3CK1BB1M4OWC441WQR1XEKTQ&client_secret=0ZRMVQMS5V1NBOPPTA1GYM3X54OGBAB1APU0LQJRKBUMPXNP&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

#### Send the GET request and examine the resutls

In [25]:
import requests
results = requests.get(url).json()


#### Lets clean the json and structure it into a pandas dataframe.

In [26]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [27]:
venues = results['response']['groups'][0]['items'][0]['venue']['name']
venues

'Brookbanks Park'

In [28]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

    
nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.venuePage.id,venue.location.postalCode
0,e-0-4e8d9dcdd5fbbbb6b3003c7b-0,0,"[{'summary': 'This spot is popular', 'type': '...",4e8d9dcdd5fbbbb6b3003c7b,Brookbanks Park,Toronto,43.751976,-79.332140,"[{'label': 'display', 'lat': 43.75197604605557...",245,CA,Toronto,ON,Canada,"[Toronto, Toronto ON, Canada]","[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",0,[],600917367,NaN
1,e-0-4e6696b6d16433b9ffff47c3-1,0,"[{'summary': 'This spot is popular', 'type': '...",4e6696b6d16433b9ffff47c3,KFC,NaN,43.754387,-79.333021,"[{'label': 'display', 'lat': 43.75438666345904...",298,CA,NaN,NaN,Canada,[Canada],"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",0,[],NaN,NaN
2,e-0-4cb11e2075ebb60cd1c4caad-2,0,"[{'summary': 'This spot is popular', 'type': '...",4cb11e2075ebb60cd1c4caad,Variety Store,29 Valley Woods Road,43.751974,-79.333114,"[{'label': 'display', 'lat': 43.75197441585782...",312,CA,Toronto,ON,Canada,"[29 Valley Woods Road, Toronto ON, Canada]","[{'id': '4bf58dd8d48988d1f9941735', 'name': 'F...",0,[],NaN,NaN
3,e-0-5921cc82e96d0c63d980640c-3,0,"[{'summary': 'This spot is popular', 'type': '...",5921cc82e96d0c63d980640c,Corrosion Service Company Limited,9-280 Hillmount Rd.,43.752432,-79.334661,"[{'label': 'display', 'lat': 43.75243208667859...",412,CA,Markham,ON,Canada,"[9-280 Hillmount Rd., Markham ON L6C 3A1, Canada]","[{'id': '5454144b498ec1f095bff2f2', 'name': 'C...",0,[],NaN,L6C 3A1


In [29]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Brookbanks Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",43.751976,-79.332140
1,KFC,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",43.754387,-79.333021
2,Variety Store,"[{'id': '4bf58dd8d48988d1f9941735', 'name': 'F...",43.751974,-79.333114
3,Corrosion Service Company Limited,"[{'id': '5454144b498ec1f095bff2f2', 'name': 'C...",43.752432,-79.334661


In [30]:
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,KFC,Fast Food Restaurant,43.754387,-79.333021
2,Variety Store,Food & Drink Shop,43.751974,-79.333114
3,Corrosion Service Company Limited,Construction & Landscaping,43.752432,-79.334661


In [31]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [32]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    #print("venue list: ", venues_list)
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    print("Near by venues: ", nearby_venues)
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
# type your answer here
NorthYork_venues = getNearbyVenues(names=NorthYork_data['Neighborhood'],
                                   latitudes=NorthYork_data['Latitude'],
                                   longitudes=NorthYork_data['Longitude']
                                  )

Near by venues:                                                     0          1          2  \
0                                          Parkwoods  43.753259 -79.329656   
1                                          Parkwoods  43.753259 -79.329656   
2                                          Parkwoods  43.753259 -79.329656   
3                                          Parkwoods  43.753259 -79.329656   
4                                   Victoria Village  43.725882 -79.315572   
5                                   Victoria Village  43.725882 -79.315572   
6                                   Victoria Village  43.725882 -79.315572   
7                                   Victoria Village  43.725882 -79.315572   
8                   Lawrence Manor, Lawrence Heights  43.718518 -79.464763   
9                   Lawrence Manor, Lawrence Heights  43.718518 -79.464763   
10                  Lawrence Manor, Lawrence Heights  43.718518 -79.464763   
11                  Lawrence Manor, Lawrence He

In [34]:
print(NorthYork_venues.shape)
NorthYork_venues.head()

(219, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Parkwoods,43.753259,-79.329656,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


In [35]:
NorthYork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
Don Mills North,4,4,4,4,4,4
Don Mills South,17,17,17,17,17,17
Downsview Central,3,3,3,3,3,3
Downsview East,2,2,2,2,2,2
Downsview Northwest,5,5,5,5,5,5
Downsview West,5,5,5,5,5,5


#### Let's find out how many unique categories can be curated from all the returned venues

In [36]:
print('There are {} uniques categories.'.format(len(NorthYork_venues['Venue Category'].unique())))

There are 90 uniques categories.


#### Analyze Each Neighborhood

In [37]:
# one hot encoding
NorthYork_onehot = pd.get_dummies(NorthYork_venues[['Venue Category']], prefix="", prefix_sep="")
print(NorthYork_onehot.shape)
NorthYork_onehot.head()

(219, 90)


,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Boutique,Bridal Shop,Burrito Place,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Electronics Store,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Gas Station,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Hockey Arena,Home Service,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Liquor Store,Luggage Store,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Park,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Spa,Sporting Goods Shop,Supermarket,Supplement Shop,Sushi Restaurant,Thai Restaurant,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [38]:
# add neighborhood column back to dataframe
NorthYork_onehot['Neighborhood'] = NorthYork_venues['Neighborhood'] 

NorthYork_onehot.head()


,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Boutique,Bridal Shop,Burrito Place,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Electronics Store,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Gas Station,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Hockey Arena,Home Service,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Liquor Store,Luggage Store,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Park,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Spa,Sporting Goods Shop,Supermarket,Supplement Shop,Sushi Restaurant,Thai Restaurant,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parkwoods
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parkwoods
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parkwoods
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parkwoods
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Victoria Village


In [39]:
# move neighborhood column to the first column
fixed_columns = [NorthYork_onehot.columns[-1]] + list(NorthYork_onehot.columns[:-1])
NorthYork_onehot = NorthYork_onehot[fixed_columns]

NorthYork_onehot.head()

,Neighborhood,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Boutique,Bridal Shop,Burrito Place,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Electronics Store,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Gas Station,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Hockey Arena,Home Service,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Liquor Store,Luggage Store,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Park,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Spa,Sporting Goods Shop,Supermarket,Supplement Shop,Sushi Restaurant,Thai Restaurant,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [40]:
NorthYork_onehot.shape

(219, 91)

#### Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [41]:
NorthYork_grouped = NorthYork_onehot.groupby('Neighborhood').mean().reset_index()
NorthYork_grouped.head()

,Neighborhood,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Boutique,Bridal Shop,Burrito Place,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Electronics Store,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Gas Station,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Hockey Arena,Home Service,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Liquor Store,Luggage Store,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Park,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Spa,Sporting Goods Shop,Supermarket,Supplement Shop,Sushi Restaurant,Thai Restaurant,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.111111,0.0,0.0,0.0,0.000000,0.0,0.055556,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.111111,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.055556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.055556,0.055556,0.055556,0.0,0.000000,0.000000,0.000000,0.0,0.055556,0.0,0.0,0.055556,0.000000,0.055556,0.000000,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.055556,0.055556,0.055556,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.055556,0.0,0.055556,0.0,0.000000,0.000000,0.0,0.055556,0.000000,0.0,0.0,0.0,0.000000
1,Bayview Village,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.250000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.250000,0.00,0.250000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.25,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
2,"Bedford Park, Lawrence Manor East",0.0,0.045455,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.045455,0.045455,0.00,0.000000,0.000000,0.090909,0.045455,0.0,0.0,0.0,0.045455,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.045455,0.045455,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.045455,0.000000,0.045455,0.00,0.0,0.0,0.0,0.045455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.045455,0.045455,0.0,0.0,0.0,0.0,0.045455,0.0,0.090909,0.0,0.090909,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.045455,0.045455,0.0,0.0,0.0,0.045455
3,Don Mills North,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.250000,0.25,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.250000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.25,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
4,Don Mills South,0.0,0.000000,0.058824,0.0,0.058824,0.0,0.0,0.000000,0.0,0.0,0.0,0.058824,0.0,0.000000,0.0,0.000000,0.058824,0.00,0.058824,0.058824,0.058824,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.058824,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.058824,0.117647,0.0,0.000000

#### Let's print each neighborhood along with the top 5 most common venues

In [42]:
num_top_venues = 5

for hood in NorthYork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = NorthYork_grouped[NorthYork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
                venue  freq
0         Coffee Shop  0.11
1                Bank  0.11
2      Ice Cream Shop  0.06
3  Frozen Yogurt Shop  0.06
4                Park  0.06


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Bank  0.25
2  Japanese Restaurant  0.25
3                 Café  0.25
4              Airport  0.00


----Bedford Park, Lawrence Manor East----
            venue  freq
0  Sandwich Place  0.09
1     Coffee Shop  0.09
2      Restaurant  0.09
3   Women's Store  0.05
4        Pharmacy  0.05


----Don Mills North----
                  venue  freq
0                   Gym  0.25
1  Caribbean Restaurant  0.25
2   Japanese Restaurant  0.25
3                  Café  0.25
4         Luggage Store  0.00


----Don Mills South----
                venue  freq
0                 Gym  0.12
1          Restaurant  0.12
2          Beer Store  0.06
3      Clothing Store  0.06
4  Chinese Resta

#### Let's put that into a pandas dataframe

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [44]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    #print("ind-->",ind)
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = NorthYork_grouped['Neighborhood']

for ind in np.arange(NorthYork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(NorthYork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Fried Chicken Joint,Sandwich Place,Pizza Place,Bridal Shop,Ice Cream Shop,Pharmacy,Park,Health Food Store
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Dog Run,Cupcake Shop,Department Store,Dessert Shop,Dim Sum Restaurant
2,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Restaurant,Women's Store,Pub,American Restaurant,Butcher,Café,Comfort Food Restaurant,Cupcake Shop
3,Don Mills North,Gym,Caribbean Restaurant,Café,Japanese Restaurant,Women's Store,Dog Run,Cupcake Shop,Department Store,Dessert Shop,Dim Sum Restaurant
4,Don Mills South,Gym,Restaurant,Grocery Store,Sandwich Place,Italian Restaurant,Dim Sum Restaurant,Coffee Shop,Chinese Restaurant,Café,Beer Store


#### Cluster Neighborhoods

In [45]:
# set number of clusters
from sklearn.cluster import KMeans
kclusters = 5

NorthYork_grouped_clustering = NorthYork_grouped.drop('Neighborhood', 1)
NorthYork_grouped_clustering.shape

(23, 90)

In [46]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(NorthYork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 4, 2, 0, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 0, 0, 2,
       2], dtype=int32)

In [47]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [48]:
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(23, 12)


,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Fried Chicken Joint,Sandwich Place,Pizza Place,Bridal Shop,Ice Cream Shop,Pharmacy,Park,Health Food Store
1,0,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Dog Run,Cupcake Shop,Department Store,Dessert Shop,Dim Sum Restaurant
2,0,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Restaurant,Women's Store,Pub,American Restaurant,Butcher,Café,Comfort Food Restaurant,Cupcake Shop
3,0,Don Mills North,Gym,Caribbean Restaurant,Café,Japanese Restaurant,Women's Store,Dog Run,Cupcake Shop,Department Store,Dessert Shop,Dim Sum Restaurant
4,0,Don Mills South,Gym,Restaurant,Grocery Store,Sandwich Place,Italian Restaurant,Dim Sum Restaurant,Coffee Shop,Chinese Restaurant,Café,Beer Store


In [49]:
NorthYork_merged = NorthYork_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
NorthYork_merged = NorthYork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

NorthYork_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,C,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,,43.753259,-79.329656,0.0,Construction & Landscaping,Food & Drink Shop,Fast Food Restaurant,Park,Discount Store,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop
1,M4A,North York,Victoria Village,,43.725882,-79.315572,0.0,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,Discount Store,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop
3,M6A,North York,"Lawrence Manor, Lawrence Heights",,43.718518,-79.464763,0.0,Clothing Store,Vietnamese Restaurant,Boutique,Coffee Shop,Miscellaneous Shop,Fried Chicken Joint,Discount Store,Cupcake Shop,Golf Course,Gas Station
7,M3B,North York,Don Mills North,,43.745906,-79.352188,0.0,Gym,Caribbean Restaurant,Café,Japanese Restaurant,Women's Store,Dog Run,Cupcake Shop,Department Store,Dessert Shop,Dim Sum Restaurant
10,M6B,North York,Glencairn,,43.709577,-79.445073,0.0,Park,Sushi Restaurant,Playground,Bakery,Italian Restaurant,Distribution Center,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store


In [50]:
NorthYork_merged=NorthYork_merged.dropna()

#### Finally, let's visualize the resulting clusters

In [51]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(len(rainbow))

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(NorthYork_merged['Latitude'], NorthYork_merged['Longitude'], NorthYork_merged['Neighborhood'], NorthYork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

5


# Cluster 1

In [52]:
NorthYork_merged.loc[NorthYork_merged['Cluster Labels'] == 0, NorthYork_merged.columns[[1] + list(range(5, NorthYork_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,-79.329656,0.0,Construction & Landscaping,Food & Drink Shop,Fast Food Restaurant,Park,Discount Store,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop
1,North York,-79.315572,0.0,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,Discount Store,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop
3,North York,-79.464763,0.0,Clothing Store,Vietnamese Restaurant,Boutique,Coffee Shop,Miscellaneous Shop,Fried Chicken Joint,Discount Store,Cupcake Shop,Golf Course,Gas Station
7,North York,-79.352188,0.0,Gym,Caribbean Restaurant,Café,Japanese Restaurant,Women's Store,Dog Run,Cupcake Shop,Department Store,Dessert Shop,Dim Sum Restaurant
10,North York,-79.445073,0.0,Park,Sushi Restaurant,Playground,Bakery,Italian Restaurant,Distribution Center,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store
13,North York,-79.340923,0.0,Gym,Restaurant,Grocery Store,Sandwich Place,Italian Restaurant,Dim Sum Restaurant,Coffee Shop,Chinese Restaurant,Café,Beer Store
27,North York,-79.363452,0.0,Golf Course,Fast Food Restaurant,Pool,Mediterranean Restaurant,Dog Run,Food Truck,Food Court,Cosmetics Shop,Cupcake Shop,Department Store
28,North York,-79.442259,0.0,Coffee Shop,Bank,Fried Chicken Joint,Sandwich Place,Pizza Place,Bridal Shop,Ice Cream Shop,Pharmacy,Park,Health Food Store
33,North York,-79.346556,0.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Japanese Restaurant,Metro Station,Bank,Restaurant,Chinese Restaurant,Mobile Phone Shop,Convenience Store
34,North York,-79.487262,0.0,Vietnamese Restaurant,Bar,Coffee Shop,Metro Station,Caribbean Restaurant,Falafel Restaurant,Miscellaneous Shop,Food & Drink Shop,Dim Sum Restaurant,Cupcake Shop


# Cluster 2

In [53]:
NorthYork_merged.loc[NorthYork_merged['Cluster Labels'] == 1, NorthYork_merged.columns[[1] + list(range(5, NorthYork_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,-79.532242,1.0,Baseball Field,Women's Store,Electronics Store,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store


# Cluster 3

In [54]:
NorthYork_merged.loc[NorthYork_merged['Cluster Labels'] == 2, NorthYork_merged.columns[[1] + list(range(5, NorthYork_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
40,North York,-79.464763,2.0,Airport,Park,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
52,North York,-79.408493,2.0,Park,Women's Store,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
66,North York,-79.400049,2.0,Park,Convenience Store,Women's Store,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store


# Cluster 4

In [55]:
NorthYork_merged.loc[NorthYork_merged['Cluster Labels'] == 3, NorthYork_merged.columns[[1] + list(range(5, NorthYork_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,North York,-79.565963,3.0,Gym,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center


# Cluster 5

In [56]:
NorthYork_merged.loc[NorthYork_merged['Cluster Labels'] == 4, NorthYork_merged.columns[[1] + list(range(5, NorthYork_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,North York,-79.495697,4.0,Home Service,Food Truck,Baseball Field,Women's Store,Dog Run,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop,Dim Sum Restaurant
